# Text-to-SQL demo using Anthropic ClaudeV2 Model in AWS Bedrock. 

### The Use Case: 
This is a very common database schema for a retail use case


![alt text](images/MySQL-Sample-Database-Schema.png "Title")

### Install dependencies

In [1]:
# !pip install SQLAlchemy==2.0.16
# !pip install pymysql==1.0.3
# !pip install cloud-sql-python-connector["pymysql"]==1.2.4
# !pip install google-cloud-aiplatform==1.30.0

In [2]:
# %pip install --no-build-isolation --force-reinstall \
#     "boto3>=1.28.57" \
#     "awscli>=1.29.57" \
#     "botocore>=1.31.57"

### Import Libraries and Set environment variables

In [6]:
import json
import os
import sys

import pymysql
import boto3
import config
import sqlalchemy

sys.path.append('config')  # Add the directory to sys.path
sys.path.append('utils')  # Add the directory to sys.path

import config  # Import the config module

# module_path = ".."
# sys.path.append(os.path.abspath(module_path))
from config import config
from utils import bedrock, print_ww

# os.environ["AWS_DEFAULT_REGION"] = "eu-central-1"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."



boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

Create new client
  Using region: eu-central-1
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.eu-central-1.amazonaws.com)


## Text-to-SQL Generative AI to get questions/answers from our database.

Let's create a general verbose style prompt and be sure to include the following sections:
* Summary of the task and provide LLM specific instructions.
* Describe the schema of the database with the tables and column types.
* Provide examples of question/answer SQL pairs.

In [3]:
_prompt = """Human: This is a task converting text into SQL statement. 
Only use the tables and columns provided. 
We will first give the dataset schema and then ask a question in text. 
You are asked to generate SQL statement. 

There are 8 tables in database classicmodels. The titles of the tables are: customers, employees, offices, orderdetails, orders, payments, productlines, products. 
Table 1 is customers, and it contains customer’s data. Its column names and types are customerNumber (Type is int), customerName (Type is varchar), contactLastName (Type is varchar), contactFirstName (Type is varchar), phone (Type is varchar), addressLine1 (Type is varchar), addressLine2 (Type is varchar), city (Type is varchar), state (Type is varchar), postalCode (Type is varchar), country (Type is varchar), salesRepEmployeeNumber (Type is int), creditLimit (Type is decimal).
Table 2 is employees, and it contains all employee information as well as the organization structure such as who reports to whom. Its column names and types are employeeNumber (Type is int), lastName (Type is varchar), firstName (Type is varchar), extension (Type is varchar), email (Type is varchar), officeCode (Type is varchar), reportsTo (Type is int), jobTitle (Type is varchar).
Table 3 is offices, and it contains sales office data. Its column names and types are officeCode (Type is varchar), city (Type is varchar), phone (Type is varchar), addressLine1 (Type is varchar), addressLine2 (Type is varchar), state (Type is varchar), country (Type is varchar), postalCode (Type is varchar), territory (Type is varchar).
Table 4 is orderdetails, and it contains sales order line items for each sales order. Its column names and types are orderNumber (Type is int), productCode (Type is varchar), quantityOrdered (Type is int), priceEach (Type is decimal), orderLineNumber (Type is smallint).
Table 5 is orders, and it contains sales orders placed by customers. Its column names and types are orderNumber (Type is int), orderDate (Type is date), requiredDate (Type is date), shippedDate (Type is date), status (Type is varchar), comments (Type is text), customerNumber (Type is int).
Table 6 is payments, and it contains payments made by customers based on their accounts. Its column names and types are customerNumber (Type is int), checkNumber (Type is varchar), paymentDate (Type is date), amount (Type is decimal).
Table 7 is productlines, and it contains a list of product line categories. Its column names and types are productLine (Type is varchar), textDescription (Type os varchar), htmlDescription (Type is mediumtext), image (Type is mediumblob).
Table 8 is products, and it contains a list of scale model cars. Its column names and types are productCode (Type is varchar), productName (Type is varchar), productLine (Type is varchar), productScale (Type is varchar), productVendor (Type is varchar), productDescription (Type is text), quantityInStock (Type is smallint), buyPrice (Type is decimal), MSRP (Type is decimal).
The primary keys are: customerNumber from Table customers, employeeNumber from Table employees,
officeCode from Table offices, orderNumber and productCode from Table orderdetails, orderNumber from Table orders, customerNumber and checkNumber from Table payments, productLine from Table productlines, productCode from Table products.
The foreign keys are: salesRepEmployeeNumber from Table customers is equivalent with employeeNumber from Table employees, officeCode from Table employees is equivalent with officeCode from table offices, reportsTo from Table employees is equivalent with employeeNumber from Table employees, customerNumber from Table orders is equivalent with customerNumber from Table customers, productLine from Table products is equivelent with productLine from Table productLines. Use foreign keys to join Tables. 

Let us take 4 example questions and turn them into a SQL statements about database classicmodels.  
Here is an example: 
The question is: How many unique customers placed an order between 1/1/2004 and 1/31/2004? 
The corresponding SQL is: SELECT count(distinct t1.customerNumber) as distinct_customers FROM orders t5 INNER JOIN customers t1 ON t5.customerNumber = t1.customerNumber WHERE t5.orderDate BETWEEN '2004-01-01' AND '2004-01-31';
Here is an example: 
The question is: What is the product name and price of the highest priced item that was ever purchased? 
The corresponding SQL is: SELECT t8.productName, t4.priceEach FROM orders t5 INNER JOIN orderdetails t4 ON t5.orderNumber = t4.orderNumber LEFT JOIN products t8 ON t4.productCode = t8.productCode ORDER BY priceEach DESC LIMIT 1;
Here is an example: 
The question is: What is the name of the customer with the highest order amount on 12/1/2003? 
The corresponding SQL is: SELECT orders.customerName, orders.total_orderAmount FROM (SELECT t5.orderNumber, t1.customerNumber, t1.customerName, SUM(t4.quantityOrdered*t4.priceEach) as total_orderAmount FROM customers t1 INNER JOIN orders t5 ON t1.customerNumber = t5.customerNumber INNER JOIN orderdetails t4 ON t5.orderNumber = t4.orderNumber WHERE t5.orderDate = '2003-12-01' GROUP BY 1,2,3) orders ORDER BY total_orderAmount DESC LIMIT 1;
Here is an example: 
The question is: How many unique orders were placed at the Paris office during the year 2004?
The corresponding SQL is: SELECT count(distinct t5.orderNumber) as orders FROM orders t5 INNER JOIN customers t1 ON t5.customerNumber = t1.customerNumber LEFT JOIN employees t2 ON t1.salesRepEmployeeNumber = t2.employeeNumber LEFT JOIN offices t3 ON t2.officeCode = t3.officeCode WHERE YEAR(t5.orderDate) = 2004 AND t3.city = 'Paris';"""

### Step 2: Create helper functions for Bedrock API and testing MySQL queries
1. **get_sql_from_gen_ai_model**: Takes the following input parameters:
    * ```prompt```: The prompt with context about the MySQL database schema and column types.
    * ```model```: It's a foundation model that generates code based on a natural language description.
2. **test_sql**: Takes a SQL query as input and attempts to run a query against a Cloud SQL MySQL database.
    * ```query```: The query to run. This can be fed directly from the previous code generation function.

In [7]:
def get_sql_from_gen_ai_model(prompt: str, question: str, model):
    
    combined_prompt = f"""{prompt}\n The test question to be answered is: {question}\n The corresponding SQL is: 
    Assistant:"""
    print(f"""Prompt to send to code gen API: \n{combined_prompt}""")

    accept = 'application/json'
    contentType = 'application/json'
    
    # Claude - Body Syntex
    body = json.dumps({
                    "prompt": combined_prompt,
                    "max_tokens_to_sample":4096,
                    "temperature":1,
                    "top_k":250,
                    "top_p":0.5,
                    "stop_sequences": ["\n\nHuman:"]
                  }) 


    response = boto3_bedrock.invoke_model(body=body, modelId=model, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())
 
    print_ww(response_body.get('completion'))

#     vertexai.init(project=project, location=region)
#     parameters = {
#         "temperature": temperature,
#         "max_output_tokens": max_output_tokens
#     }
#     code_gen_model = CodeGenerationModel.from_pretrained(model)
    
#     response = code_gen_model.predict(
#         prefix = combined_prompt,
#         **parameters
#     )

    return (response_body.get('completion'))
    



def test_sql(query: str):
    
    # function to return the database connection object
    def getconn():
        # RDS connection info
        username = config.db_user
        password = config.db_password
        host = config.db_host
        port = config.db_port
        database = config.db_name

        # create the connection
        # conn = pymysql.connect(host=host, user=username, password=password, db=database, port=port)
        conn = pymysql.connect(host="database-1.cg8brvavjrza.eu-central-1.rds.amazonaws.com",
                               user="admin", password="Pass1234", db="classicmodels", port=3306)
        return conn

    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "mysql+pymysql://",
        creator=getconn,
    )

    # connect to connection pool
    with pool.connect() as db_conn:
        # query and fetch ratings table
        results = db_conn.execute(sqlalchemy.text(f"{query};")).fetchall()
        # show results
        output_list = []
        for row in results:
            output_list.append(row)
         
    return output_list

In [9]:
easy_text_to_sql = """What were sales for the year 2005 by state filtered by customers based in USA?"""

easy_verbose_sql_query = get_sql_from_gen_ai_model(
                      prompt = _prompt,
                      question = easy_text_to_sql,
                      model = "anthropic.claude-v2")
print(easy_verbose_sql_query)

easy_verbose_test_results = test_sql(easy_verbose_sql_query)
print("Query results: ", easy_verbose_test_results)

Prompt to send to code gen API: 
Human: This is a task converting text into SQL statement. 
Only use the tables and columns provided. 
We will first give the dataset schema and then ask a question in text. 
You are asked to generate SQL statement. 

There are 8 tables in database classicmodels. The titles of the tables are: customers, employees, offices, orderdetails, orders, payments, productlines, products. 
Table 1 is customers, and it contains customer’s data. Its column names and types are customerNumber (Type is int), customerName (Type is varchar), contactLastName (Type is varchar), contactFirstName (Type is varchar), phone (Type is varchar), addressLine1 (Type is varchar), addressLine2 (Type is varchar), city (Type is varchar), state (Type is varchar), postalCode (Type is varchar), country (Type is varchar), salesRepEmployeeNumber (Type is int), creditLimit (Type is decimal).
Table 2 is employees, and it contains all employee information as well as the organization structure su